In [ ]:
import control.matlab as cnt
import numpy as np
from matplotlib import rcParams
from utils import create_output_dir, plot_bode, plot_response, plot_responses

from sippy_unipi import functionset as fset
from sippy_unipi import system_identification
from sippy_unipi.datasets import load_sample_siso

rcParams["figure.figsize"] = (9.0, 5.0)
output_dir = create_output_dir("test_armax.ipynb")
seed = 0
np.random.seed(seed)

In [ ]:
n_samples = 401
ts = 1.0
time, Ysim, Usim, g_sys, Yerr, Uerr, h_sys, Ytot, Utot = load_sample_siso(
    n_samples, ts, seed=seed
)

In [ ]:
fig = plot_responses(
    time,
    [Usim, Uerr, Utot],
    [Ysim, Yerr, Ytot],
    ["u", "e", ["u", "e"]],
)

fig.savefig(output_dir + "/responses.png")

## Perform system identification from collected data

In [ ]:
sys_id = system_identification(
    Ytot,
    Usim,
    "ARMAX",
    *((1, 5), (1, 5), (1, 5), (1, 13)),
    IC="BIC",
    centering="MeanVal",
    max_iter=1000,
)

## Check that output of the identified system is consistent

In [ ]:
Y_id1, time, Xsim = cnt.lsim(sys_id.G, Usim, time)
Y_hid1, time, Xsim = cnt.lsim(sys_id.H, Uerr, time)
Y_idTot = Y_id1 + Y_hid1

In [ ]:
fig = plot_response(
    time,
    Usim,
    [Ytot, Y_idTot],
    legends=[["U"], ["system", "armax"]],
    titles=[
        "Input, identification data (Switch probability=0.08)",
        "Output (identification data)",
    ],
)
fig.savefig(output_dir + "/system_consistency.png")

# Validation of the identified system: 
## Generate new time series for input and noise

In [ ]:
switch_probability = 0.07  # [0..1]
input_range = [0.5, 1.5]
U_valid, _, _ = fset.GBN_seq(n_samples, switch_probability, scale=input_range)
white_noise_variance = [0.01]
e_valid = fset.white_noise_var(U_valid.size, white_noise_variance)[0]

## Compute time responses for true system with new inputs

In [ ]:
Yvalid1, time, Xsim = cnt.lsim(g_sys, U_valid, time)
Yvalid2, time, Xsim = cnt.lsim(h_sys, e_valid, time)
Ytotvalid = Yvalid1 + Yvalid2

## Compute time responses for identified system with new inputs

In [ ]:
y_idvalid1, time, Xsim = cnt.lsim(sys_id.G, U_valid, time)
y_idvalid2, time, Xsim = cnt.lsim(sys_id.H, e_valid, time)
y_idtotvalid = y_idvalid1 + y_idvalid2

## Check responses are almost equal

In [ ]:
fig = plot_response(
    time,
    Usim,
    [Ytotvalid, y_idtotvalid],
    legends=[["U"], ["system", "armax"]],
    titles=[
        "Input, identification data (Switch probability=0.07)",
        "Output (identification data)",
    ],
)
fig.savefig(output_dir + "/system_validation.png")

In [ ]:
W_V = np.logspace(-3, 4, num=701)
for tf in ["G", "H"]:
    syss_tfs = [
        locals()[f"{tf.lower()}_sys"],
        getattr(sys_id, tf),
    ]
    mags, fis, oms = zip(*[cnt.bode(sys, W_V) for sys in syss_tfs])

    fig = plot_bode(
        oms[0],
        mags,
        fis,
        ["system"],
    )
    fig.savefig(output_dir + f"/bode_{tf}.png")